# Imports

In [1]:
import json
import os
import bz2
import io
from bz2 import BZ2File
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

import seaborn as sns
sns.set_context('paper')
sns.set_style("white")
sns.set(rc = {'figure.figsize':(12,9)})

import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

# Charger le dataset

In [2]:
df_cleaned = pd.read_json('cleaned.json.bz2', compression = 'bz2')
df_cleaned['journal'] = df_cleaned['journal'].astype('category')
df_cleaned.head()

,id,journal,date,page,ppage,text,length
0,0,JDG,1978-12-29,1,NaN,Algérie : l'arbitrage sans arbitre Pour un Eta...,3777
1,1,JDG,1978-12-29,3,NaN,ALORS QUE LA FOULE ENTERRE SES DERNIERES VICTI...,3401
100,100,JDG,1978-06-13,1,NaN,La Ruhr chinoise s'ouvre à l'Occident Gilbert ...,6017
10000,10000,JDG,1978-10-25,11,NaN,Selon le président du directoire de la Banque ...,4345
100000,100000,GDL,1976-10-05,2,NaN,TÉLÉVISION-RADIO SELECTION-TV Antenne 2 20.30 ...,7976


In [3]:
df_cleaned = df_cleaned.drop(columns = 'id')

In [4]:
df_cleaned.index = np.arange(len(df_cleaned))

# Nettoyage ciblé

In [5]:
def keywords_count(df, keywords):
    """ Select text that contains certain keywords and count the latter 
    
    Inputs:
        df(pandas DataFrame): contains all articles in the 'text' column
        keywords(list<str>): list of keywords to search
        
    Output:
        counts(list<list<int>>): count of all keywords in all articles
    """
    
    counts, k = [], []
    
    for keyword in keywords:
        k.append(keyword.lower())
    
    for ind, row in df.iterrows():
        
        counts_ = []
        for k_ in k:
            counts_.append(len(re.findall(k_, row['text'].lower())))
        
        counts.append(counts_)
                
    return counts

In [6]:
garbage = ['tableau hebdomadaire',
           'aetoa',
           'sopacx',
           'cotées en suisse',
           'new york stock exch',
           'new york nys',
           'volvo',
           'nyse',
           'nestlé',
           'heineken',
           'ericsson',
           'stock exch',
           'siemens',
           'yorkst',
           'quebec hydro',
           'aetn a',
           'raytheo',
           'euro devises',
           'royal dutch']

In [7]:
counts_garbage = keywords_count(df_cleaned, garbage)
counts_garbage = np.asarray(counts_garbage).T

In [8]:
for i in range(len(garbage)):
    df_cleaned['keyword_' + garbage[i]] = counts_garbage[i]

In [9]:
for i in range (len(garbage)):
    df_cleaned = df_cleaned[df_cleaned['keyword_' + garbage[i]] == 0]

In [10]:
len(df_cleaned)

440317

# Isoler le corpus pour la dernière couche de nettoyage ciblée

In [11]:
keywords = ['pétrol', 
            'opep', 
            'organisation des pays exportateurs de pétrole',
            'shell',
            'tamoil',
            'petrol',
            'petroleum',
            'texaco',
            'standard oil',
            'exxon',
            'gulf oil',
            'ats',
            'afp',
            'reuter',
            'reuther']

In [12]:
counts = keywords_count(df_cleaned, keywords)
counts = np.asarray(counts).T

In [13]:
for i in range(len(keywords)):
    df_cleaned['keyword_' + keywords[i]] = counts[i]

In [14]:
df_cleaned['keywords_petrole'] = df_cleaned['keyword_pétrol'] + df_cleaned['keyword_petrol']
df_cleaned['keywords_OPEP'] = df_cleaned['keyword_opep'] + \
    df_cleaned['keyword_organisation des pays exportateurs de pétrole']
df_cleaned['keywords_compagnies'] = df_cleaned['keyword_shell'] + \
    df_cleaned['keyword_tamoil'] + df_cleaned['keyword_petroleum'] + df_cleaned['keyword_texaco'] + \
    df_cleaned['keyword_standard oil'] + df_cleaned['keyword_exxon'] + df_cleaned['keyword_gulf oil']
df_cleaned['keywords'] = df_cleaned['keywords_petrole'] + df_cleaned['keywords_OPEP'] + \
    df_cleaned['keywords_compagnies']

df_cleaned['depeches'] = df_cleaned['keyword_afp'] + df_cleaned['keyword_ats'] + \
df_cleaned['keyword_reuther'] + df_cleaned['keyword_reuter']


In [15]:
df_corpus = df_cleaned[df_cleaned['keywords'] > 0]

# Nettoyage post-filtrage

In [16]:
df_depeches = df_corpus[df_corpus['depeches'] > 0]
df_not_depeches = df_corpus[df_corpus['depeches'] == 0]

In [17]:
len(df_depeches)

17853

In [18]:
def compare_texts(txt1, txt2):
    ''' Compare texts to check for duplicates. Proceeds in two steps (to spare ressources). First
    by aligning roughly the two texts and then by verifiying more precisely this alignment.
    
    Inputs:
        txt1(str): First text to compare
        txt2(str): Second text to compare
        
    Output:
        (bool): returns True if the texts are duplicates, else False
    '''
    
    identical = False
    recover = 0
        
    ### First step is a rough assessment ###
    
    text1 = txt1.split(' ')
    text2 = txt2.split(' ')
    
    if (len(text1) > 20) and (len(text2) > 20):
        text1, text2 = text1[:20], text2[:20]
        
    else:
        min_ = min(len(text1), len(text2))
        text1, text2 = text1[:min_], text2[:min_]
        
    if set(text1) == set(text2):
        return True
    
    differentials, grade = [], 0
    
    # estimate identity 
    for i in range(len(text1)):
        for j in range(len(text2)):
            if text1[i] == text2[j]:
                grade += 1
                differentials.append(j - i)
    
    _ = np.unique(differentials, return_counts = True)
    
    if len(_[1]) == 0:
        return False
    
    else:
            
        ### Second step is a finer assessment ###
        if (grade >= 12) and (np.argmax(_[1].tolist()) >= 12): 
            
            if (len(text1) > 50) and (len(text2) > 50):
                text1, text2 = text1[:50], text2[:50]
                
                differentials, grade = [], 0
                
                for i in range(len(text1)):
                    for j in range(len(text2)):
                        if text1[i] == text2[j]:
                            grade += 1
                            differentials.append(j - i)
                            
                _ = np.unique(differentials, return_counts = True)
                
                if (grade >= 40) and (np.argmax(_[1].tolist()) >= 40):
                    return True

            else:
                return True
    
        else:
            return False

In [19]:
def check_duplicates(df):
    ''' Search for duplicates in all articles'''

    first_day = min(df_depeches['date'])
    last_day = max(df_depeches['date'])
    day = first_day
    
    duplicates, checked = [], []

    while day <= last_day:
        border = [day - timedelta(days = 7), day + timedelta(days = 7)]
        df_1 = df[df['date'] == day] # ?
        df_2 = df[df['date'] >= border[0]]
        df_2 = df_2[df_2['date'] <= border[1]]
        
        for ind_1, row_1 in df_1.iterrows():
            for ind_2, row_2 in df_2.iterrows():
                
                if not(ind_2 in checked):
                    if (ind_1 != ind_2) and compare_texts(row_1['text'], row_2['text']):
                        duplicates.append([ind_1, ind_2])
            
            checked.append(ind_1)
        
        day += timedelta(days = 1)
        
    return duplicates
            

In [20]:
df_ats = df_corpus[df_corpus['keyword_ats'] > 0]
df_reuter = df_corpus[df_corpus['keyword_reuter'] > 0]
df_reuther = df_corpus[df_corpus['keyword_reuther'] > 0]
df_afp = df_corpus[df_corpus['keyword_afp'] > 0]

In [21]:
duplicates = check_duplicates(df_reuther)
duplicates += check_duplicates(df_reuter)

In [22]:
duplicates += check_duplicates(df_afp)

In [23]:
duplicates += check_duplicates(df_ats)

In [24]:
to_drop = []

for i in range(len(duplicates)):
    if not(duplicates[i][0] in to_drop) and not(duplicates[i][1] in to_drop):
        to_drop.append(duplicates[i][0])

In [25]:
len(to_drop)

1410

In [26]:
df_corpus_unique = df_corpus.drop(to_drop)

In [27]:
len(df_corpus)

22549

In [28]:
len(df_corpus_unique)

21139

In [29]:
df_corpus_unique.to_json('corpus_unique.json.bz2', compression = 'bz2')

In [30]:
df_corpus_unique = pd.read_json('corpus_unique.json.bz2', compression = 'bz2')
df_corpus_unique['journal'] = df_corpus_unique['journal'].astype('category')
df_corpus_unique.index = np.arange(len(df_corpus_unique))


In [31]:
df_ = df_corpus_unique.drop(columns = ['keyword_tableau hebdomadaire', 'keyword_aetoa', 'keyword_sopacx',
                                      'keyword_cotées en suisse', 'keyword_new york stock exch',
                                      'keyword_new york nys'])

# Création des sous-corpus

In [32]:
keywords = ['pétrol', 
            'opep', 
            'organisation des pays exportateurs de pétrole',
            'shell',
            'tamoil',
            'petrol',
            'petroleum',
            'texaco',
            'standard oil',
            'exxon',
            'gulf oil',
            'ats',
            'afp',
            'reuter',
            'reuther']

In [33]:
counts = keywords_count(df_, keywords)
counts = np.asarray(counts).T

In [34]:
for i in range(len(keywords)):
    df_['keyword_' + keywords[i]] = counts[i]

In [35]:
df_['keywords_petrole'] = df_['keyword_pétrol'] + df_['keyword_petrol']
df_['keywords_OPEP'] = df_['keyword_opep'] + \
    df_['keyword_organisation des pays exportateurs de pétrole']
df_['keywords_compagnies'] = df_['keyword_shell'] + \
    df_['keyword_tamoil'] + df_['keyword_petroleum'] + df_['keyword_texaco'] + \
    df_['keyword_standard oil'] + df_['keyword_exxon'] + df_['keyword_gulf oil']
df_['keywords'] = df_['keywords_petrole'] + df_['keywords_OPEP'] + \
    df_['keywords_compagnies']

df_['depeches'] = df_['keyword_afp'] + df_['keyword_ats'] + df_['keyword_reuther'] + df_['keyword_reuter']

In [36]:
df_corpus = df_[df_['keywords'] > 0]

# Sauvegarde des sous-corpus

In [37]:
df_corpus_OPEP = df_corpus[df_corpus['keywords_OPEP'] > 0]
df_corpus_petrole = df_corpus[df_corpus['keywords_petrole'] > 0]
df_corpus_compagnies = df_corpus[df_corpus['keywords_compagnies'] > 0]

In [38]:
df_depeches = df_corpus[df_corpus['depeches'] > 0]
df_not_depeches = df_corpus[df_corpus['depeches'] == 0]

In [39]:
df_corpus.to_json('corpus.json.bz2', compression = 'bz2')
df_corpus_OPEP.to_json('corpus_OPEP.json.bz2', compression = 'bz2')
df_corpus_petrole.to_json('corpus_petrole.json.bz2', compression = 'bz2')
df_corpus_compagnies.to_json('corpus_compagnies.json.bz2', compression = 'bz2')

In [16]:
df_cleaned.to_json('total.json.bz2', compression = 'bz2')

In [40]:
df_depeches.to_json('corpus_depeches.json.bz2', compression = 'bz2')
df_not_depeches.to_json('corpus_not_depeches.json.bz2', compression = 'bz2')

In [14]:
df_corpus = pd.read_json('corpus.json.bz2', compression = 'bz2')
df_corpus_OPEP = pd.read_json('corpus_OPEP.json.bz2', compression = 'bz2')
df_corpus_petrole = pd.read_json('corpus_petrole.json.bz2', compression = 'bz2')
df_corpus_compagnies = pd.read_json('corpus_compagnies.json.bz2', compression = 'bz2')
df_depeches = pd.read_json('corpus_depeches.json.bz2', compression = 'bz2')
df_not_depeches = pd.read_json('corpus_not_depeches.json.bz2', compression = 'bz2')

In [17]:
df_total = pd.read_json('total.json.bz2', compression = 'bz2')
df_total['journal'] = df_total['journal'].astype('category')
df_total.index = np.arange(len(df_total))

In [15]:
df_corpus['journal'] = df_corpus['journal'].astype('category')
df_corpus_OPEP['journal'] = df_corpus_OPEP['journal'].astype('category')
df_corpus_petrole['journal'] = df_corpus_petrole['journal'].astype('category')
df_corpus_compagnies['journal'] = df_corpus_compagnies['journal'].astype('category')

df_corpus.index = np.arange(len(df_corpus))
df_corpus_OPEP.index = np.arange(len(df_corpus_OPEP))
df_corpus_petrole.index = np.arange(len(df_corpus_petrole))
df_corpus_compagnies.index = np.arange(len(df_corpus_compagnies))

# Export au format Iramuteq

In [62]:
df_corpus_OPEP.head()

,journal,date,page,ppage,text,length,keyword_volvo,keyword_nyse,keyword_nestlé,keyword_heineken,...,keyword_gulf oil,keyword_ats,keyword_afp,keyword_reuter,keyword_reuther,keywords_petrole,keywords_OPEP,keywords_compagnies,keywords,depeches
0,JDG,1974-01-07,2,NaN,"Pétrole : "" (déjà 50000 la crise continue chôm...",3465,0,0,0,0,...,0,2,1,0,0,6,5,0,11,3
1,JDG,1971-03-03,16,NaN,A l'est de Suez La Grande-Bretagne quittera le...,3335,0,0,0,0,...,0,4,1,0,0,0,1,0,1,5
2,JDG,1971-09-23,5,NaN,Après une séance unique de trois heures Fin de...,1275,0,0,0,0,...,0,0,1,0,0,2,4,0,6,1
3,JDG,1971-07-10,5,NaN,. Conférence extraordinaire des. pays exportat...,2560,0,0,0,0,...,0,1,3,0,0,15,9,0,24,4
4,JDG,1971-01-22,5,NaN,Les compagnies pétrclières créent un bureau pe...,913,0,0,0,0,...,0,0,1,0,0,2,1,0,3,1


In [63]:
def formate_text(df):
    
    text = ''
    for row in df.iterrows():
        text += '**** *' + row[1][0] + ' *' + \
            str(row[1][1])[:10] + ' *' + str(row[1][2]) + ' *' + \
            str(row[1][5]) + '\n' + row[1][4] + '\n'
        
    return text

In [64]:
def iramuteq_export(df, filename):
    
    text = formate_text(df)
    file = open(filename,'w') 
    file.write(text)
    file.close()
    
    return True

In [65]:
iramuteq_export(df_corpus, 'corpus.txt')
iramuteq_export(df_corpus_OPEP, 'corpus_opep.txt')
iramuteq_export(df_corpus_petrole, 'corpus_petrole.txt')
iramuteq_export(df_corpus_compagnies, 'corpus_compagnies.txt')
iramuteq_export(df_depeches, 'depeches.txt')
iramuteq_export(df_not_depeches, 'not_depeches.txt')

True